# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

spaceship.dropna(inplace=True)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
from sklearn.preprocessing import StandardScaler

features = spaceship.select_dtypes("float64")
target = spaceship["Transported"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

X_train = pd.DataFrame(X_train, columns=features.columns)
X_test = pd.DataFrame(X_test, columns=features.columns)

scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

X_train_norm = pd.DataFrame(X_train_norm, columns=X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns=X_test.columns)

print(X_train_norm.head())
print(X_test_norm.head())

        Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck
0 -0.267707    -0.350087   0.694479     -0.302537  0.300959  0.256747
1  0.492810    -0.350087   1.252114      2.468142  2.231549 -0.263876
2  0.354535    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
3  0.561948    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
4 -0.475120    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
        Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck
0  0.008845    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
1 -1.097362    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
2  1.460741    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
3 -1.581327    -0.350087  -0.286513     -0.302537 -0.273653 -0.264728
4  0.561948    -0.022472  -0.284754      0.702399 -0.272758 -0.206786


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [10]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train_norm, y_train)

LogisticRegression()

- Evaluate your model

In [11]:
log_reg.score(X_test_norm, y_test)

0.7806354009077155

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [12]:
parameter_grid = {"max_depth": [10, 50],
                  "min_samples_split": [4, 16],
                  "max_leaf_nodes": [250, 100],
                  "max_features": ["sqrt", "log2"]}

- Run Grid Search

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import time
import scipy.stats as st

# We create an instance or our machine learning model
dt = DecisionTreeRegressor(random_state=123)

# We need to set this two variables to be able to compute a confidence interval
confidence_level = 0.95
folds = 10

# Now we need to create an intance of the GridSearchCV class
gs = GridSearchCV(dt, param_grid=parameter_grid, cv=folds, verbose=10) # Here the "cv" allows you to define the number of folds to use.

start_time = time.time()
gs.fit(X_train_norm, y_train)
end_time = time.time()

print("\n")
print(f"Time taken to find the best combination of hyperparameters among the given ones: {end_time - start_time: .4f} seconds")
print("\n")


print(f"The best combination of hyperparameters has been: {gs.best_params_}")
print(f"The R2 is: {gs.best_score_: .4f}")

results_gs_df = pd.DataFrame(gs.cv_results_).sort_values(by="mean_test_score", ascending=False)

#print(results_df.head())
gs_mean_score = results_gs_df.iloc[0,-3]
gs_sem = results_gs_df.iloc[0,-2] / np.sqrt(10)

gs_tc = st.t.ppf(1-((1-confidence_level)/2), df=folds-1)
gs_lower_bound = gs_mean_score - ( gs_tc * gs_sem )
gs_upper_bound = gs_mean_score + ( gs_tc * gs_sem )

print(f"The R2 confidence interval for the best combination of hyperparameters is: \
    ({gs_lower_bound: .4f}, {gs_mean_score: .4f}, {gs_upper_bound: .4f}) ")

#display(results_df)

# Let's store the best model
best_model = gs.best_estimator_

# Now is time evaluate the model in the test set
y_pred_test = best_model.predict(X_test_norm)
y_pred_test = best_model.predict(X_test_norm)

y_pred_test = best_model.predict(X_test_norm)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 1/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.259 total time=   0.0s
[CV 2/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 2/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.295 total time=   0.0s
[CV 3/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 3/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.253 total time=   0.0s
[CV 4/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 4/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.201 total time=   0.0s
[CV 5/10; 1/16] START max_depth=10, max_features=sqrt

- Evaluate your model

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

print("\n")
print(f"Test MAE: {mean_absolute_error(y_pred_test, y_test): .4f}")
print(f"Test MSE: {mean_squared_error(y_pred_test, y_test): .4f}")
print(f"Test RMSE: {root_mean_squared_error(y_pred_test, y_test): .4f}")
print(f"Test R2 score:  {best_model.score(X_test_norm, y_test): .4f}")
print("\n")



Test MAE:  0.3173
Test MSE:  0.1717
Test RMSE:  0.4144
Test R2 score:   0.3131


